<div class="alert alert-block alert-danger">
    
# Initial Processing
</div>

In [ ]:
## Import the libraries
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score, make_scorer,confusion_matrix, matthews_corrcoef, f1_score, ConfusionMatrixDisplay
#import stats
import re
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from itertools import chain
import collections
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge, ElasticNet, LinearRegression
from sklearn.svm import SVR
from pathlib import Path
import datetime
import random as python_random
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import xgboost
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from ctgan import CTGAN
from PyALE import ale
import shap
from sklearn.model_selection import StratifiedKFold
from sdmetrics.reports.single_table import QualityReport
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.metrics import precision_score, recall_score,f1_score
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import silhouette_score
import matplotlib.image as pltimg
import itertools
from sklearn.metrics import davies_bouldin_score
from itertools import combinations
#from validclust import ValidClust
from sklearn.cluster import KMeans
#import winsound


<div class="alert alert-block alert-warning">
    
## Extract data from Employment History
</div>

In [ ]:
## Extract the data from csv file (Employment History file)
df2 = pd.read_csv('../data/raw/EmploymentHistorycsv.csv', engine = 'python', encoding = 'latin1')

In [ ]:
## Specify the key words to extract rows according to that
relevant_words = ['nurse', 'nurs', 'care', 'health', 'hosp', 'hospital', 'nursing', 'med', 'medical', 'infirmary', 'nhs', 
                  'midwife', 'midwifery', 'hiv', 'residence', 'residential', 'elder', 'patient','diabetes mellitus',
                 'gastroenteritis', 'disease', 'cardio', 'heart', 'red cross']

In [ ]:
## Convert the values in department column to lower case
df2['department'] = df2['department'].str.lower()

## Fill the null values with random word
df2['department'] = df2['department'].fillna('culo')
df2['responsibilities'] = df2['responsibilities'].fillna('culo')
df2['position'] = df2['position'].fillna('culo')

In [ ]:
## Change the data type of department, position and responsibilities to string
df2['department'] = df2['department'].astype('str') 
df2['position'] = df2['position'].astype('str')
df2['responsibilities'] = df2['responsibilities'].astype('str')

## Concatenate the position, department and responsibilities and loading the value in history column
df2['history'] = df2[['position', 'department', 'responsibilities']].agg(' '.join, axis=1)

## Group the details in the dataframe by 'candidateid'
dfex = df2.groupby('candidateid',as_index= False).agg(' '.join)

In [ ]:
## Function to check if the candidate has health related experience
def check_exp(df2, col1):
    lista1 = []
    for i in df2[col1].to_list():
        if any(word in i for word in relevant_words):
            lista1.append(1)
        else:
            lista1.append(0)
        
    return lista1

In [ ]:
## To take the details of those candidates that has worked in healthcare field
checked = check_exp(dfex, 'history')
dfex['health_exp'] = checked
df_to_export = dfex[['candidateid', 'health_exp']]

In [ ]:
## Export the result generated to excel file
df_to_export.to_excel('../data/processed/healthcare_experiences.xlsx')



<div class="alert alert-block alert-warning">
    
## Select relevant features
</div>

In [ ]:
## Load data from the CandidateApplicationsData file and healthcare_experiences file
dfexp = pd.read_excel('../data/processed/healthcare_experiences.xlsx')
dfcad = pd.read_csv('../data/raw//CandidateApplicationsDatacsv.csv', engine = 'python', encoding = 'latin1')

In [ ]:
## Retrieve the column names from CandidateApplicationsData file
col_names = dfcad.loc[[1]].values.flatten().tolist()
dfcad = dfcad.iloc[2:]
dfcad.columns = col_names

In [ ]:
## Rename the 'candidate id' column to 'candidateid'
dfcad.rename(columns = {'candidate id':'candidateid'}, inplace = True)

In [ ]:
## Merge the csv files based on 'candidateid'
dfcad.candidateid = dfcad.candidateid.astype('int64')
df_merge = pd.merge(dfexp, dfcad, on='candidateid')
df = df_merge

In [ ]:
## The list of columns that are to be considered for analysis and modelling
lista = ['candidateid', 'Contracted hours',
         'length of service in months','Current Employee [1=YES / 0=No]','Applied Through (Channel Type)',
         'Applied Through (Channel)', 'Job Title', 'Advertised Salary', ' Distance Candidate is from Job location (KM) ',
        'Certification', 'All','All', 'Accounts Skills',  'Admin Skills', 'Care Assistant', 'Chef Skills', 'FlexForce', 
         'General Skills', 'Health and Social Care ', 'Home Manager Skills', 'Kitchen & Domestic Skills', 'Maintenance Skills', 
         'Nurse Skills', 'Qualifications', 'Specialist Area', 'Team Leader Skills', 'Total Skills on CV','Socail Demographic Code',
        'Count of Care homes within 15 min drive', 'Count of Care home Beds  within 15 min drive', 'Average age of the care homes within 15 min drive', 
         'Count of Care Homes within walking distance (<500 meters) from candidates address',
         'Sum of care home beds within walking distance from candidates address', 'Count of bus stops walking distance from candidates address ',
        '1. Owned','1. Owned','3. Private rented','4. Living rent free','Employee: full-time', 'Employee: part-time',
         'Self-employed', 'Unemployed (economically active)', 'Full-time student (economically active)','Total economically active',
        'Total economically inactive','No car or van in household','Total cars or vans','Total Population', 'Males',
         'Females', 'Residents in households', 'Residents in communal establishments', 'Schoolchildren or full-time students aged 4+ at their non term-time address',
        'All people aged 16+','Single (never married or registered a same-sex civil partnership)',
         'Married', 'In a registered same-sex civil partnership', 'Separated (but still legally married or in a same-sex civil partnership)', 'health_exp',
         'Divorced (or formerly in a same-sex civil partnership which is now legally dissolved)', 'Widowed (or surviving partner from a same-sex civil partnership)']

listadef = list(set(lista))

In [ ]:
## Select relevant features
df = df[listadef]

In [ ]:
## Eliminate repeated columns
df = df.loc[:,~df.apply(lambda x: x.duplicated(),axis=1).all()].copy()

In [ ]:
## To load the data to file
df.to_excel('../data/processed/finalfinal.xlsx')

<div class="alert alert-block alert-danger">
    
# Data Cleaning
</div>



<div class="alert alert-block alert-warning">
    
## Deal with duplicates
</div>

In [ ]:
## To read data from excel file
df = pd.read_excel('../data/processed/finalfinal.xlsx')

In [ ]:
## To find the number of entries for a particular candidate
values = df['candidateid'].value_counts(dropna=False).keys().tolist()
counts = df['candidateid'].value_counts(dropna=False).tolist()
value_dict = dict(zip(values, counts))

In [ ]:
## Dropping the duplicate rows from the dataframe, only keeping the last row for the duplicate values 
df = df.drop_duplicates(subset=['candidateid'], keep='last')

In [ ]:
## Writing the dataframe content to excel file
df.to_excel('../data/processed/final_nodup.xlsx')

In [ ]:
## To get the candidate entries that are greater than 1
new_data = {k: v for k, v in value_dict.items() if v > 1}



<div class="alert alert-block alert-warning">
    
## Missing values
</div>

In [ ]:
## To read the excel file
dff = pd.read_excel('../data/processed/final_nodup.xlsx')

In [ ]:
## Columns to remove from the dataframe
cols_to_remove = ['In a registered same-sex civil partnership','Residents in communal establishments','Chef Skills',
                 'Care Assistant', 'Team Leader Skills', 'Maintenance Skills', 'Qualifications','Accounts Skills',
                  'Applied Through (Channel)', 'Applied Through (Channel Type)']

dff.drop(cols_to_remove, axis = 1, inplace = True)
# dff.drop(['Unnamed: 0.1', 'Unnamed: 0_x', 'Unnamed: 0_y'], axis = 1, inplace = True)
# dff.drop(['bonus'], axis = 1, inplace = True)

In [ ]:
## Finding the percentage of values that are null in the dataframe
percent_missing = dff.isna().sum() * 100 / len(dff)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})

In [ ]:
missing_value_df[missing_value_df['percent_missing']>5]

In [ ]:
## Dropping the null values
df_nomis = dff.dropna()



<div class="alert alert-block alert-warning">
    
## Cleaning salary
</div>

In [ ]:
## Accessing the dataset
emp_data = pd.read_excel('../data/processed/final_nodup.xlsx')

In [ ]:
#Preprocessing the salary column

#Function to calculate per hour salary
def calculate_average(range_str,contracted_hr):
    range_str = str(range_str).rstrip(".")

    
    if range_str.startswith("-") or range_str.endswith("-"):
      range_str = range_str.strip("-")    
    if ("-" in range_str):
      
      start,end = range_str.split("-")
      if (float(start)<=20 and float(end)>=1000):
        range_str = float(start)
      else:
        avg = (float(start)+float(end))/2
        range_str = avg
      
    if (range_str!=''  and float(range_str)>20000):
      if float(contracted_hr) !=0:
        sal = float(range_str)/(float(contracted_hr)*52)
        range_str = sal
      else:
        sal = float(range_str)/2080
        range_str = sal
    if (range_str==''):
      range_str = 0
    return float(range_str)
   

r = emp_data.loc[:,['candidateid','Advertised Salary','Contracted hours']]

#Converting to lowercase
r["Advertised Salary"] = r["Advertised Salary"].str.lower()

#Removing space, to, upto, plus, comma
r = r.replace({' 0 ': '-'}, regex=True)
r = r.replace({' ': ''}, regex=True)
r = r.replace({'upto': '','to£':'-£',',plus':',+','up':'','/hr':'',',£':'+£'}, regex=True)
r = r.replace({'to': '-'}, regex=True)
r = r.replace({',': '','plus':'+','--':'-'}, regex=True)

#Remove characters after equivalent
r[['Advertised Salary', 'equ']] = r['Advertised Salary'].str.split('equ', expand=True)
r.drop('equ', axis=1, inplace=True)

#Remove charaters after NMC registered
r[['Advertised Salary', 'nmc']] = r['Advertised Salary'].str.split('nmcregistered', expand=True)
r.drop('nmc', axis=1, inplace=True)

#Remove charaters after pro rata
r[['Advertised Salary', 'pro']] = r['Advertised Salary'].str.split('prorata', expand=True)
r.drop('pro', axis=1, inplace=True)

#Remove all the characters except +,digit,dot,-
pattern = r"[^+,\-/.\d]" 
r["Advertised Salary"] = r["Advertised Salary"].apply(lambda x: re.sub(pattern, "", str(x)))

#Separate the bonus and salary
r[['Advertised Salary', 'bonus']] = r['Advertised Salary'].str.split('+', expand=True)

#Replacing -- with -
r = r.replace({'--':'-','/':'-'}, regex=True)

#Calculate the per hour salary
r["Advertised Salary"] = r.apply(lambda row: calculate_average(row["Advertised Salary"], row["Contracted hours"]), axis=1)

# calculate the median of the salary, ignoring zeros
median = np.median(r.loc[r['Advertised Salary'] != 0, 'Advertised Salary'])

# replace zeros with the median
r.loc[r['Advertised Salary'] == 0, 'Advertised Salary'] = median

r = r.drop(['Contracted hours'], axis=1)

In [ ]:
## Writing the dataframe after preprocessing the salary column 
r.to_csv('../data/processed/salary_clean.csv', encoding='utf-8')


<div class="alert alert-block alert-warning">
    
## Outliers

</div>

In [ ]:
#Rerun the below parts as the salary_clean.csv data was modified
## Load the file with preprocessed advertised salary
df2 = pd.read_csv('../data/processed/salary_clean.csv')

## Dropping the 'Advertised Salary' from df_nomis, and merge the preprocessed 'Advertised Salary' to the dataframe df  
df_nomis.drop('Advertised Salary', axis = 1, inplace = True)
df = pd.merge(df_nomis, df2, on='candidateid')

In [ ]:
df = df[(df['Advertised Salary']<38) & (df['Total Population']<600)]

In [ ]:
## Sorting the data frame according to the salary value
f = sorted(df['Advertised Salary'].to_list(), reverse = True)

In [ ]:
## Function to count the number of outliers in the dataframe
def count_outlier_values_per_row(dfexample, t):
    freq_remain = []
    
    for var in dfexample.columns:
        Q1 = dfexample[var].quantile(0.25)
        Q3 = dfexample[var].quantile(0.75)
        IQR = Q3 - Q1
        remain = dfexample.candidateid[(dfexample[var]>(Q1 - t * IQR)) & (dfexample[var]<(Q3 + t * IQR))].tolist() #get candidateids of rows that do not contain outliers in that variable
        freq_remain.append(remain)

    unzipped = list(chain.from_iterable(freq_remain))
    counter = collections.Counter(unzipped) #count frequency of appearnce of each candidateid (same as number of columns for which that candidate is not an outlier)
    dfexample["number_outliers"] = df.apply(lambda row: counter[row.candidateid], axis=1) # create new feature with the generated dict
    return dfexample

In [ ]:
## Removing the bonus column and unnamed column
df.drop(['Unnamed: 0.1', 'Unnamed: 0_x', 'Unnamed: 0_y'], axis = 1, inplace = True)
df.drop(['bonus'], axis = 1, inplace = True)

In [ ]:
## Columns that can have outliers
cols_outliers = ['candidateid','Total economically inactive', 'Full-time student (economically active)','Single (never married or registered a same-sex civil partnership)',
'Widowed (or surviving partner from a same-sex civil partnership)',
'Separated (but still legally married or in a same-sex civil partnership)',
'Count of bus stops walking distance from candidates address ','Contracted hours',
'Schoolchildren or full-time students aged 4+ at their non term-time address','Unemployed (economically active)',
'Sum of care home beds within walking distance from candidates address',
'Count of Care Homes within walking distance (<500 meters) from candidates address','Self-employed','Employee: full-time']

t=1.5
for var in cols_outliers:
    Q1 = df[var].quantile(0.25)
    Q3 = df[var].quantile(0.75)
    IQR = Q3 - Q1
    df = df[(df[var]>(Q1 - t * IQR)) & (df[var]<(Q3 + t * IQR))]

In [ ]:
## Writing the dataframe after removing the outliers to excel file
df.to_excel('../data/processed/df_nomis_noout.xlsx')

In [ ]:
## Writing the dataframe after removing the outliers to csv file
df.to_csv('../data/processed/df_nomis_noout.csv')



<div class="alert alert-block alert-warning">
    
## Categorize string values
</div>

In [ ]:
## Read the excel file with no outliers
df = pd.read_excel('../data/processed/df_nomis_noout.xlsx')

In [ ]:
## Mentioning the 
strings = ['Socail Demographic Code', 'Job Title', 'All.1']
# distance --> impute by distribution

In [ ]:
## Assign values for each category
var = 'Socail Demographic Code'
df[var] = df[var].astype('category')
mapping_soc = dict( zip(df[var],df[var].cat.codes ) )
df[var] = df[var].map(mapping_soc)

In [ ]:
var = 'Job Title'
df[var] = df[var].astype('category')
mapping_job = dict( zip(df[var],df[var].cat.codes ) )
df[var] = df[var].map(mapping_job)

In [ ]:
np.save('../data/processed/mapping_soc.npy', mapping_soc) 
np.save('../data/processed/mapping_job.npy', mapping_job) 

In [ ]:
var = 'All.1'

#Replacing the columns () with value 'Not Found' as 0 and changing it into float

df[var] = df[var].replace({'Not Found':0})

df[var] = df[var].astype(float)

In [ ]:
df.to_csv('../data/processed/prejake_final.csv')

In [ ]:
df[' Distance Candidate is from Job location (KM) '].dtype

In [ ]:
#Replace 'Distance not found' with NAN, converting into float

df[' Distance Candidate is from Job location (KM) '] = df[' Distance Candidate is from Job location (KM) '].replace({'Distance not found':np.nan})
df[' Distance Candidate is from Job location (KM) '] = df[' Distance Candidate is from Job location (KM) '].astype(float)

#Take the candidate information where the job location is less than 50 km

df = df[df[' Distance Candidate is from Job location (KM) ']<50]


In [ ]:
#Imputing the ' Distance Candidate is from Job location (KM) ' using the distribution

f = np.array(df[' Distance Candidate is from Job location (KM) ']).reshape(-1, 1)

imputer = KNNImputer(n_neighbors=3, weights='uniform', metric='nan_euclidean')
imputer.fit(f)
Xtrans = imputer.transform(f)

df[' Distance Candidate is from Job location (KM) '] = Xtrans

In [ ]:
df.to_csv('../data/processed/final.csv')



<div class="alert alert-block alert-warning">
    
# Oversampling
</div>

In [ ]:
df = pd.read_csv('../data/processed/final.csv')

In [ ]:
aa = pd.read_csv('../data/processed/final.csv')

In [ ]:
#defining the binary, categorical & continuous variables with the column names of binary, categorical and continuous values from the dataframe.

binary = ['Current Employee [1=YES / 0=No]','health_exp']
categorical = ['Job Title']
continuous = ['4. Living rent free',
 'Employee: part-time',
 'No car or van in household',
 'Total economically active',
 'Total economically inactive',
 'Males',
 'Widowed (or surviving partner from a same-sex civil partnership)',
 'Count of Care home Beds  within 15 min drive',
 'Full-time student (economically active)',
 'Total Population',
 'Kitchen & Domestic Skills',
 'Self-employed',
 'Females',
 'Total cars or vans',
 'Single (never married or registered a same-sex civil partnership)',
 'Separated (but still legally married or in a same-sex civil partnership)',
 'Health and Social Care ',
 'Nurse Skills',
 'Count of bus stops walking distance from candidates address ',
 'Certification',
 'General Skills',
 '3. Private rented',
 'All people aged 16+',
 'Average age of the care homes within 15 min drive',
 'Socail Demographic Code',
 'Admin Skills',
 'Employee: full-time',
 'Contracted hours',
 'Married',
 'length of service in months',
 'Home Manager Skills',
 'Count of Care homes within 15 min drive',
 'Schoolchildren or full-time students aged 4+ at their non term-time address',
 'Divorced (or formerly in a same-sex civil partnership which is now legally dissolved)',
 ' Distance Candidate is from Job location (KM) ',
 '1. Owned',
 'Unemployed (economically active)',
 'Sum of care home beds within walking distance from candidates address',
 'Residents in households',
 'candidateid',
 'Specialist Area',
 'Total Skills on CV',
 'Count of Care Homes within walking distance (<500 meters) from candidates address',
 'FlexForce',
 'All',
 'All.1',
 'Advertised Salary']

In [ ]:
mapj = np.load('../data/processed/mapping_job.npy',allow_pickle=True)

In [ ]:
#Retriving the candidate details based on the current employment status and job title

carers = df[(df['Current Employee [1=YES / 0=No]'] == 0) & (df['Job Title'] == 0)]
nurses = df[(df['Current Employee [1=YES / 0=No]'] == 0) & (df['Job Title'] == 1)]
seniors = df[(df['Current Employee [1=YES / 0=No]'] == 0) & (df['Job Title'] == 2)]

In [ ]:
#Function to oversample the values in each class

def oversample_class(df_x_train_real, percentage = 0.25):
    n_epochs=20
    batch_size=40
    seed_value = 1
#     torch.manual_seed(seed_value)
    np.random.seed(seed_value)

    v_column_names = df_x_train_real.columns.to_list()

    n_synthetic_samples = int(df_x_train_real.shape[0] * percentage)

    oversampler_model = CTGAN(epochs=n_epochs, batch_size=batch_size, verbose=True, cuda=False)
    oversampler_model.fit(df_x_train_real, v_column_names)

    df_x_train_synthetic = oversampler_model.sample(n_synthetic_samples)
    df_x_train_synthetic.to_csv('../data/processed/synthetic_senior_carers.csv')
    return df_x_train_synthetic

In [ ]:
#Function to find the quality metrics of data

def compute_quality_metrics(df_real, df_synthetic, list_categorical_vars, return_score=False):
    dict_aux = {}
    for var_name in df_real.columns.values:
        if var_name in list_categorical_vars:
            dict_aux[var_name] = {'type': 'categorical'}
        else:
            dict_aux[var_name] = {'type': 'numerical'}
    dict_metadata = {'fields': dict_aux}

    report = QualityReport()
    report.generate(df_real, df_synthetic, dict_metadata)
    df_metrics = report.get_details(property_name='Column Shapes')

    fig1 = report.get_visualization(property_name='Column Shapes')
    plt.savefig('../reports/figures/colshapes_report.png')
    fig1.show()

    fig = report.get_visualization(property_name='Column Pair Trends')
    plt.savefig('../reports/figures/colpairs_report.png')
    fig.show()

    if return_score:
        qscore = df_metrics.iloc[:, -1].values
        return qscore

In [ ]:
nurses = nurses.rename(columns = {'Count of Care home Beds  within 15 min drive' : 'Beds15drive' ,
       'Sum of care home beds within walking distance from candidates address' :  'BedsWalking',
       'Count of bus stops walking distance from candidates address ': 'BusStops',
       'Single (never married or registered a same-sex civil partnership)':'Single',
       'All':'Vocational_qual', 'All.1' : 'Academic_qual', 'Home Manager Skills':'HomeManagSkills', 'Employee: full-time': 'FTEmp',
       'Contracted hours': 'ContHours', 'Certification':'Certif', 'Kitchen & Domestic Skills': 'DomSkills',
       'Total economically active':'Active', 'Nurse Skills': 'NurseSkills', 'Admin Skills':'AdminSkills',
       'Total Skills on CV':'TotSKills',
       'Average age of the care homes within 15 min drive':'AvgAge',
       'Health and Social Care ': 'SocialCare', 'Married':'Married', 'Current Employee [1=YES / 0=No]': 'CurrEmp',
       'Unemployed (economically active)':'Unemp',
       'Full-time student (economically active)':'Student',
       'length of service in months':'LoS', 'All people aged 16+' : 'Age16+',
       ' Distance Candidate is from Job location (KM) ':  'Dist2Job',
       'Count of Care Homes within walking distance (<500 meters) from candidates address': 'Homes<500',
       'Total cars or vans':'TotCars', 'Total Population':'Population', 'Total economically inactive':'Inactive',
       'health_exp':'HExp', 'Count of Care homes within 15 min drive':'Homes15drive', '1. Owned':'Owned',
       '4. Living rent free':'RentFree',
       'Schoolchildren or full-time students aged 4+ at their non term-time address':'Children',
       'No car or van in household':'NoCar',
       'Widowed (or surviving partner from a same-sex civil partnership)':'Widows',
       'General Skills':'GenSkills', 'Females':'Females',
       'Separated (but still legally married or in a same-sex civil partnership)':'Separated',
       'FlexForce':'FlexForce', 'Employee: part-time':'PartTime', 'Socail Demographic Code':'DemoCode',
       'candidateid':'CandidateID', '3. Private rented':'PrivRent', 'Males':'Males', 'Specialist Area':'SpecArea',
       'Residents in households':'Residents',
       'Divorced (or formerly in a same-sex civil partnership which is now legally dissolved)':'Divorced',
       'Job Title':'Job', 'Self-employed':'SelfEmp', 'Advertised Salary':'AdvSalary'})

In [ ]:
nurses_os = oversample_class(nurses, percentage = 0.25)

In [ ]:
a = compute_quality_metrics(nurses, nurses_os, categorical, return_score=True)

In [ ]:
seniors = seniors.rename(columns = {'Count of Care home Beds  within 15 min drive' : 'Beds15drive' ,
       'Sum of care home beds within walking distance from candidates address' :  'BedsWalking',
       'Count of bus stops walking distance from candidates address ': 'BusStops',
       'Single (never married or registered a same-sex civil partnership)':'Single',
       'All':'Vocational_qual', 'All.1' : 'Academic_qual', 'Home Manager Skills':'HomeManagSkills', 'Employee: full-time': 'FTEmp',
       'Contracted hours': 'ContHours', 'Certification':'Certif', 'Kitchen & Domestic Skills': 'DomSkills',
       'Total economically active':'Active', 'Nurse Skills': 'NurseSkills', 'Admin Skills':'AdminSkills',
       'Total Skills on CV':'TotSKills',
       'Average age of the care homes within 15 min drive':'AvgAge',
       'Health and Social Care ': 'SocialCare', 'Married':'Married', 'Current Employee [1=YES / 0=No]': 'CurrEmp',
       'Unemployed (economically active)':'Unemp',
       'Full-time student (economically active)':'Student',
       'length of service in months':'LoS', 'All people aged 16+' : 'Age16+',
       ' Distance Candidate is from Job location (KM) ':  'Dist2Job',
       'Count of Care Homes within walking distance (<500 meters) from candidates address': 'Homes<500',
       'Total cars or vans':'TotCars', 'Total Population':'Population', 'Total economically inactive':'Inactive',
       'health_exp':'HExp', 'Count of Care homes within 15 min drive':'Homes15drive', '1. Owned':'Owned',
       '4. Living rent free':'RentFree',
       'Schoolchildren or full-time students aged 4+ at their non term-time address':'Children',
       'No car or van in household':'NoCar',
       'Widowed (or surviving partner from a same-sex civil partnership)':'Widows',
       'General Skills':'GenSkills', 'Females':'Females',
       'Separated (but still legally married or in a same-sex civil partnership)':'Separated',
       'FlexForce':'FlexForce', 'Employee: part-time':'PartTime', 'Socail Demographic Code':'DemoCode',
       'candidateid':'CandidateID', '3. Private rented':'PrivRent', 'Males':'Males', 'Specialist Area':'SpecArea',
       'Residents in households':'Residents',
       'Divorced (or formerly in a same-sex civil partnership which is now legally dissolved)':'Divorced',
       'Job Title':'Job', 'Self-employed':'SelfEmp', 'Advertised Salary':'AdvSalary'})

In [ ]:
seniors_os = oversample_class(seniors, percentage = 0.25)

In [ ]:
b = compute_quality_metrics(seniors, seniors_os, categorical, return_score=True)

In [ ]:
#carers = carers.rename(columns = {'Count of Care home Beds  within 15 min drive' : 'Beds15drive' ,
       'Sum of care home beds within walking distance from candidates address' :  'BedsWalking',
       'Count of bus stops walking distance from candidates address ': 'BusStops',
       'Single (never married or registered a same-sex civil partnership)':'Single',
       'All':'Vocational_qual', 'All.1' : 'Academic_qual', 'Home Manager Skills':'HomeManagSkills', 'Employee: full-time': 'FTEmp',
       'Contracted hours': 'ContHours', 'Certification':'Certif', 'Kitchen & Domestic Skills': 'DomSkills',
       'Total economically active':'Active', 'Nurse Skills': 'NurseSkills', 'Admin Skills':'AdminSkills',
       'Total Skills on CV':'TotSKills',
       'Average age of the care homes within 15 min drive':'AvgAge',
       'Health and Social Care ': 'SocialCare', 'Married':'Married', 'Current Employee [1=YES / 0=No]': 'CurrEmp',
       'Unemployed (economically active)':'Unemp',
       'Full-time student (economically active)':'Student',
       'length of service in months':'LoS', 'All people aged 16+' : 'Age16+',
       ' Distance Candidate is from Job location (KM) ':  'Dist2Job',
       'Count of Care Homes within walking distance (<500 meters) from candidates address': 'Homes<500',
       'Total cars or vans':'TotCars', 'Total Population':'Population', 'Total economically inactive':'Inactive',
       'health_exp':'HExp', 'Count of Care homes within 15 min drive':'Homes15drive', '1. Owned':'Owned',
       '4. Living rent free':'RentFree',
       'Schoolchildren or full-time students aged 4+ at their non term-time address':'Children',
       'No car or van in household':'NoCar',
       'Widowed (or surviving partner from a same-sex civil partnership)':'Widows',
       'General Skills':'GenSkills', 'Females':'Females',
       'Separated (but still legally married or in a same-sex civil partnership)':'Separated',
       'FlexForce':'FlexForce', 'Employee: part-time':'PartTime', 'Socail Demographic Code':'DemoCode',
       'candidateid':'CandidateID', '3. Private rented':'PrivRent', 'Males':'Males', 'Specialist Area':'SpecArea',
       'Residents in households':'Residents',
       'Divorced (or formerly in a same-sex civil partnership which is now legally dissolved)':'Divorced',
       'Job Title':'Job', 'Self-employed':'SelfEmp', 'Advertised Salary':'AdvSalary'})

In [ ]:
#carers_os = oversample_class(carers, percentage = 0.25)


In [ ]:
#c = compute_quality_metrics(carers, carers_os, categorical, return_score=True)

In [ ]:
# merge synthetic data from nurses and seniors to original data from carer
final_nurses = pd.concat([nurses, nurses_os])
final_seniors = pd.concat([seniors, seniors_os])

In [ ]:
# save final csv
file_os = pd.concat([final_nurses, final_seniors,carers])

In [ ]:
current_employees = df[(df['Current Employee [1=YES / 0=No]'] == 1)]

In [ ]:
final_file_oversamples = pd.concat([file_os, current_employees])

In [ ]:
final_file_oversamples.to_csv('../data/processed/data.csv')

<div class="alert alert-block alert-danger">
    
# Statistical Analysis
   
</div>

In [ ]:
df = pd.read_csv('../data/processed/data.csv')

In [ ]:
df.drop(['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
df.columns

In [ ]:
#defining the binary, categorical & continuous variables with the column names of binary, categorical and continuous values from the dataframe

binary = ['Current Employee [1=YES / 0=No]','health_exp']
categorical = ['Job Title']
continuous = ['4. Living rent free',
 'Employee: part-time',
 'No car or van in household',
 'Total economically active',
 'Total economically inactive',
 'Males',
 'Widowed (or surviving partner from a same-sex civil partnership)',
 'Count of Care home Beds  within 15 min drive',
 'Full-time student (economically active)',
 'Total Population',
 'Kitchen & Domestic Skills',
 'Self-employed',
 'Females',
 'Total cars or vans',
 'Single (never married or registered a same-sex civil partnership)',
 'Separated (but still legally married or in a same-sex civil partnership)',
 'Health and Social Care ',
 'Nurse Skills',
 'Count of bus stops walking distance from candidates address ',
 'Certification',
 'General Skills',
 '3. Private rented',
 'All people aged 16+',
 'Average age of the care homes within 15 min drive',
 'Socail Demographic Code',
 'Admin Skills',
 'Employee: full-time',
 'Contracted hours',
 'Married',
 'length of service in months',
 'Home Manager Skills',
 'Count of Care homes within 15 min drive',
 'Schoolchildren or full-time students aged 4+ at their non term-time address',
 'Divorced (or formerly in a same-sex civil partnership which is now legally dissolved)',
 ' Distance Candidate is from Job location (KM) ',
 '1. Owned',
 'Unemployed (economically active)',
 'Sum of care home beds within walking distance from candidates address',
 'Residents in households',
 'candidateid',
 'Specialist Area',
 'Total Skills on CV',
 'Count of Care Homes within walking distance (<500 meters) from candidates address',
 'FlexForce',
 'All',
 'All.1',
 'Advertised Salary']

In [ ]:
#Retriving the candidate details based on the current employment status and job title

carers = df[(df['Current Employee [1=YES / 0=No]'] == 0) & (df['Job Title'] == 0)]
nurses = df[(df['Current Employee [1=YES / 0=No]'] == 0) & (df['Job Title'] == 1)]
seniors = df[(df['Current Employee [1=YES / 0=No]'] == 0) & (df['Job Title'] == 2)]

In [ ]:
seniors.shape



<div class="alert alert-block alert-warning">
    
## Correlation analysis
</div>

In [ ]:
binary = ['Current Employee [1=YES / 0=No]','health_exp']
categ = ['Job Title']
continuous = ['4. Living rent free',
 'Employee: part-time',
 'No car or van in household',
 'Total economically active',
 'Total economically inactive',
 'Males',
 'Widowed (or surviving partner from a same-sex civil partnership)',
 'Count of Care home Beds  within 15 min drive',
 'Full-time student (economically active)',
 'Total Population',
 'Kitchen & Domestic Skills',
 'Self-employed',
 'Females',
 'Total cars or vans',
 'Single (never married or registered a same-sex civil partnership)',
 'Separated (but still legally married or in a same-sex civil partnership)',
 'Health and Social Care ',
 'Nurse Skills',
 'Count of bus stops walking distance from candidates address ',
 'Certification',
 'General Skills',
 '3. Private rented',
 'All people aged 16+',
 'Average age of the care homes within 15 min drive',
 'Socail Demographic Code',
 'Admin Skills',
 'Employee: full-time',
 'Contracted hours',
 'Married',
 'length of service in months',
 'Home Manager Skills',
 'Count of Care homes within 15 min drive',
 'Schoolchildren or full-time students aged 4+ at their non term-time address',
 'Divorced (or formerly in a same-sex civil partnership which is now legally dissolved)',
 ' Distance Candidate is from Job location (KM) ',
 '1. Owned',
 'Unemployed (economically active)',
 'Sum of care home beds within walking distance from candidates address',
 'Residents in households',
 'candidateid',
 'Specialist Area',
 'Total Skills on CV',
 'Count of Care Homes within walking distance (<500 meters) from candidates address',
 'FlexForce',
 'All',
 'All.1',
 'Advertised Salary']

In [ ]:
# chi-square test
def Cramers_v(df, col1, col2):
    a = pd.crosstab(df[col1], df[col2], margins = False)

    X2 = stats.chi2_contingency(np.array(a), correction=False)[0]
    N = np.sum(np.array(a))
    minimum_dimension = min(np.array(a).shape)-1
    result = np.sqrt((X2/N) / minimum_dimension)

    return np.sqrt((X2/N) / minimum_dimension)

# Function to study correlation between vairables, using different coefficients for different types of variables;
def correlation_all(df, numerical, categ, binary):

    corr_vals = []
    categorical = categ + binary
    variables = categorical + continuous
    listvar1 = []

    for var1 in variables:
        listvar2 = []
        for var2 in variables:

            if var1 in continuous and var2 in continuous:
                corr, _ = stats.pearsonr(df[var1], df[var2])
                listvar2.append(corr)

            if var1 in continuous and var2 in categorical:
                pbc, pval = stats.pointbiserialr(df[var2], df[var1])
                listvar2.append(pbc)

            if var1 in categorical and var2 in continuous:
                pbc, pval = stats.pointbiserialr(df[var2], df[var1])
                listvar2.append(pbc) 

            if var1 in categorical and var2 in categorical:
                listvar2.append(Cramers_v(df, var1, var2))

        listvar1.append(listvar2)

    corrdf = pd.DataFrame(listvar1, columns = variables)
    corrdf['Var'] = variables
    corrdf.set_index('Var', inplace = True)
    corrdf.update(corrdf.abs())  
    return corrdf, corr_vals

In [ ]:
df = carers
corr_vals = []
categorical = categ + binary
variables = categorical + continuous
listvar1 = []

for var1 in variables:
    listvar2 = []
    for var2 in variables:
        
        if var1 in continuous and var2 in continuous:
            corr, _ = stats.pearsonr(df[var1], df[var2])
            listvar2.append(corr)

        if var1 in continuous and var2 in categorical:
            pbc, pval = stats.pointbiserialr(df[var2], df[var1])
            listvar2.append(pbc)

        if var1 in categorical and var2 in continuous:
            pbc, pval = stats.pointbiserialr(df[var2], df[var1])
            listvar2.append(pbc) 

        if var1 in categorical and var2 in categorical:
            listvar2.append(Cramers_v(df, var1, var2))
        
    listvar1.append(listvar2)
    
corrdfcarers = pd.DataFrame(listvar1, columns = variables)
corrdfcarers['Var'] = variables
corrdfcarers.set_index('Var', inplace = True)
corrdfcarers.update(corrdfcarers.abs())

sns.heatmap(corrdfcarers, annot=False)
plt.savefig('../reports/figures/corr_carers.pdf', bbox_inches = 'tight')

In [ ]:
df = nurses
corr_vals = []
categorical = categ + binary
variables = categorical + continuous
listvar1 = []

for var1 in variables:
    listvar2 = []
    for var2 in variables:
        
        if var1 in continuous and var2 in continuous:
            corr, _ = stats.pearsonr(df[var1], df[var2])
            listvar2.append(corr)

        if var1 in continuous and var2 in categorical:
            pbc, pval = stats.pointbiserialr(df[var2], df[var1])
            listvar2.append(pbc)

        if var1 in categorical and var2 in continuous:
            pbc, pval = stats.pointbiserialr(df[var2], df[var1])
            listvar2.append(pbc) 

        if var1 in categorical and var2 in categorical:
            listvar2.append(Cramers_v(df, var1, var2))
        
    listvar1.append(listvar2)
    
corrdfnurses = pd.DataFrame(listvar1, columns = variables)
corrdfnurses['Var'] = variables
corrdfnurses.set_index('Var', inplace = True)
corrdfnurses.update(corrdfnurses.abs())

sns.heatmap(corrdfnurses, annot=False)
plt.savefig('../reports/figures/corr_nurses.pdf', bbox_inches = 'tight')

<div class="alert alert-block alert-danger">
    
# Clustering
</div>

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

In [ ]:
df = pd.read_csv('../data/processed/data.csv')

In [ ]:
kmeans = KMeans(n_clusters=2)

y = kmeans.fit_predict(df[['length of service in months']])

In [ ]:
from scipy.stats import f_oneway
f_statistic, p_value = f_oneway(df['length of service in months'], y)

print("P-Value:", p_value)

In [ ]:
df['Cluster'] = y

In [ ]:
from sklearn.datasets import make_blobs
X, y = make_blobs(n_samples=df.shape[0], centers=2, n_features=1)

In [ ]:
categorical = ['health_exp', 'Job Title']
numerical = [var for var in df.columns if var not in categorical]
print(categorical)
print()
print(numerical)

In [ ]:
df_0 = df[df['Cluster'] == 0]
df_1 = df[df['Cluster'] == 1]
print(max(df_0['length of service in months']), min(df_0['length of service in months']))
print(max(df_1['length of service in months']), min(df_1['length of service in months']))
cluster_dfs = [df_0, df_1]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.distplot(df_0['length of service in months'], hist=False, kde_kws=dict(linewidth=5))
sns.distplot(df_1['length of service in months'], hist=False, kde_kws=dict(linewidth=5))
plt.xlim(-10, 100)
plt.axvline(x = 26, color = 'g', label = 'axvline - full height', linestyle='--', lw=3)
plt.xlabel("Length of Service in Months")
plt.title("Density plots with the threshold")
plt.savefig('density_clusters.png', dpi=300)
plt.show()

In [ ]:
sns.distplot(df['length of service in months'], hist=False, kde_kws=dict(linewidth=5))
plt.xlim(-10, 100)
plt.axvline(x = 26, color = 'g', label = 'axvline - full height', linestyle='--', lw=3)
plt.xlabel("Length of Service in Months")
plt.title("Density plot with the threshold")
plt.show()

<div class="alert alert-block alert-danger">
    
# Modeling
</div>

In [ ]:
import math

In [ ]:
#scaling the test and train data

def scale_features(x_train, x_test):
    scaler = MinMaxScaler()
    x_train_raw = x_train.copy()
    x_test_raw = x_test.copy()
    scaler.fit(x_train)
    x_train = pd.DataFrame(data = scaler.transform(x_train), columns = list(x_train_raw.columns))
    x_test = pd.DataFrame(data = scaler.transform(x_test), columns = list(x_train_raw.columns))
    return x_train, x_test, x_train_raw, x_test_raw

In [ ]:
def plot_corresponding_ale_numerical_feature(df_ale_numerical, df_ale_numerical_raw, var_name, estimator_name, itera, job, flag_save_figure=True):

    df_ale_numerical = df_ale_numerical.reset_index()
    df_ale_numerical_raw = df_ale_numerical_raw.reset_index()
    first_std = df_ale_numerical.iloc[0, 1]
    df_ale_numerical['lowerCI_95%'] = df_ale_numerical['lowerCI_95%'].fillna(first_std)
    df_ale_numerical['upperCI_95%'] = df_ale_numerical['upperCI_95%'].fillna(first_std)

    fig, ax = plt.subplots(1, 1, figsize=(5, 3.1))
    ax.plot(df_ale_numerical_raw[var_name], df_ale_numerical['eff'], ls='solid', color='blue')
    ax.fill_between(df_ale_numerical_raw[var_name], df_ale_numerical['lowerCI_95%'], df_ale_numerical['upperCI_95%'], label='95% CI', alpha=0.3, color='grey', lw=2)
    ax.set_xlabel(var_name, fontsize=12)

    plt.xticks(fontsize=11)
    plt.yticks(fontsize=11)
    plt.legend()
    plt.grid(alpha=0.5, linestyle='--')
    fig.tight_layout()
    
    if '<' in var_name:
        var_name = var_name.replace("<", "")

    plt.savefig('../reports/figures/{}_{}_ale_{}_seed_{}.pdf'.format(job,estimator_name,var_name,itera), bbox_inches = 'tight')
    plt.clf()
    plt.close()

In [ ]:
def plot_corresponding_ale_categorical_feature(df_ale_cat, var_name, estimator_name, itera,job, flag_save_figure=True):

    df_ale_cat = df_ale_cat.reset_index()
    first_std = df_ale_cat.iloc[0, 1]
    df_ale_cat['lowerCI_95%'] = df_ale_cat['lowerCI_95%'].fillna(first_std)
    df_ale_cat['upperCI_95%'] = df_ale_cat['upperCI_95%'].fillna(first_std)

    fig, ax = plt.subplots(1, 1, figsize=(5, 3.1))
    ax.bar(df_ale_cat[var_name], df_ale_cat['eff'], width=0.4, color='blue', alpha=0.0)
    ax.set_ylabel('', fontsize=16)
    ax2 = ax.twinx()
    ax2.bar(df_ale_cat[var_name], df_ale_cat['size'], width=0.4, color='blue', alpha=0.3)
    ax2.set_ylabel('Size', fontsize=16)
    ax.plot([df_ale_cat.iloc[0, 0], df_ale_cat.iloc[1, 0]],[df_ale_cat.iloc[0, 1], df_ale_cat.iloc[1, 1]],'--', marker='o', color='black')

    for index, row in df_ale_cat.iterrows():
        p_inf = [row[var_name], row['lowerCI_95%']]
        p_center = [row[var_name], row['eff']]
        yerr = math.sqrt((p_center[0] - p_inf[0])**2 + (p_center[1] - p_inf[1])**2)
        ax.errorbar(row[var_name], row['eff'], yerr, fmt='o', lw=2, capsize=4, color='black')

    ax.set_xlabel(var_name, fontsize=12)
    ax.set_xticks([0, 1])
    plt.grid(alpha=0.5, linestyle='--')
    fig.tight_layout()
    
    if '<' in var_name:
        var_name = var_name.replace("<", "") 
    
    plt.savefig('../reports/figures/{}_{}_ale_{}_seed_{}.pdf'.format(job,estimator_name,var_name,itera), bbox_inches = 'tight')
    plt.clf()
    plt.close()

In [ ]:
def plot_ale_features(df_x_test, model_estimator, df_x_test_raw, list_vars_numerical, list_vars_categorical, estimator_name, itera, job):

    for var_numerical in list_vars_numerical:
        df_ale_numerical = ale(X=df_x_test,model=model_estimator,feature=[var_numerical],plot=False,feature_type="continuous",grid_size=10,include_CI=True,C=0.95)
        df_ale_numerical_raw = ale(X=df_x_test_raw,model=model_estimator,feature=[var_numerical],plot=False,feature_type="continuous",grid_size=10,include_CI=True,C=0.95)
        plot_corresponding_ale_numerical_feature(df_ale_numerical, df_ale_numerical_raw, var_numerical,estimator_name, itera, job)

    for var_categorical in list_vars_categorical:
        ale_discr = ale(X=df_x_test,model=model_estimator,feature=[var_categorical],feature_type="discrete",plot=False,grid_size=10,include_CI=True,C=0.95)
        plot_corresponding_ale_categorical_feature(ale_discr, var_categorical,estimator_name, itera, job)

In [ ]:
def study_conf(job, classifier):
    
    df_metrics = pd.read_excel("../data/processed/metrics_{}.xlsx".format(job))
    dfconf = df_metrics[df_metrics['Classifier']== classifier]
    
    a = dfconf['cm'][dfconf['Iteration']==1].values[0].split()
    b = dfconf['cm'][dfconf['Iteration']==2].values[0].split()
    c = dfconf['cm'][dfconf['Iteration']==3].values[0].split()
    d = dfconf['cm'][dfconf['Iteration']==4].values[0].split()
    e = dfconf['cm'][dfconf['Iteration']==5].values[0].split()

    confusiona = np.array([[int(a[0][2:]),int(a[1][:-1]) ], [int(a[3]),int(a[4][:-2])]])
    confusionb = np.array([[int(b[0][2:]),int(b[1][:-1]) ], [int(b[3]),int(b[4][:-2])]])
    confusionc = np.array([[int(c[0][2:]),int(c[1][:-1]) ], [int(c[3]),int(c[4][:-2])]])
    confusiond = np.array([[int(d[0][2:]),int(d[1][:-1]) ], [int(d[3]),int(d[4][:-2])]])
    confusione = np.array([[int(e[0][2:]),int(e[1][:-1]) ], [int(e[3]),int(e[4][:-2])]])

    final_conf = confusiona+confusionb+confusionc+confusiond+confusione

    df_cm = pd.DataFrame(final_conf, index = [i for i in ['Succesful', 'Unsuccesful']],
                      columns = [i for i in ['Succesful', 'Unsuccesful']])
#     sns.set(font_scale=3)
    plt.figure(figsize = (10,7))
    b = sns.heatmap(df_cm, annot=True, cmap = 'Blues',fmt="d",annot_kws={'size': 30})
    b.set_yticklabels(['Succesful', 'Unsuccesful'],size = 30)
    b.set_xticklabels(['Succesful', 'Unsuccesful'],size = 30)
    plt.savefig('../reports/figures/{}_{}_confusionmatrix.pdf'.format(job, classifier),bbox_inches = 'tight')

In [ ]:
TOT = 1e-4
MAX_ITERS = 20000
TEST_SIZE_PROPORTION = 0.2
VALIDATION_SIZE_PROPORTION = 0.2

## To get the best possible hyper parameter

def get_hyperparams(classifier_name: str, num_features, seed_value: int = 4242): 
    MAX_ITERS = 20000
    dict_param_grid_classifier = {}
    classifier = None

    if classifier_name == 'svm':
        dict_param_grid_classifier = {'C': np.linspace(0.01, 10, 10),'gamma': ['auto'], 'kernel': ['rbf', 'linear', 'poly', 'sigmoid'],
                                     'class_weight':['balanced'],'probability':[True]} 
        classifier = SVC(max_iter=MAX_ITERS, random_state=seed_value)

    elif classifier_name == 'dt':
        dict_param_grid_classifier = {'max_depth': np.arange(1, 30, 1), 'min_samples_split': np.arange(2, 50, 2),
                                      'class_weight':['balanced']}
        classifier = DecisionTreeClassifier(random_state=seed_value)

    elif classifier_name == 'rf':
        dict_param_grid_classifier = {'max_depth': np.arange(1, 30, 3), 'n_estimators': np.arange(20, 50, 5)
                                      ,'min_samples_split': np.arange(2, 25, 5),'class_weight':['balanced']}
        classifier = RandomForestClassifier(n_jobs=-1, random_state=seed_value)

    elif classifier_name == 'knn':
        dict_param_grid_classifier = {'n_neighbors': np.arange(1, 20, 1)}
        classifier = KNeighborsClassifier()
  
    elif classifier_name == 'lr':
        dict_param_grid_classifier = {'penalty': ['l2', 'none'],'class_weight':['balanced']}
        classifier = LogisticRegression(multi_class='multinomial', n_jobs=-1, random_state=seed_value)
 
    list_dicts_params_learning_curve = []

    for key, value in dict_param_grid_classifier.items():
        dict_param_lrc = {'param_name': key, 'param_range': value}
        list_dicts_params_learning_curve.append(dict_param_lrc)

    return classifier, dict_param_grid_classifier, list_dicts_params_learning_curve

In [ ]:
def two_proportion_test(len1, prop1, len2, prop2, variable):
    
    n1 = len1
    n2 = len2
    n = n1+n2
    p1 = prop1
    p2 = prop2
    x1 = p1*n1
    x2 = p2*n2
    p_pool = (x1+x2)/n
    se = np.sqrt(p_pool*(1-p_pool)*(1/n1+1/n2))
    z_statistic = (p1-p2)/se
    p_value = 2*stats.norm().cdf(-1*np.abs(z_statistic))

    return p_value
        
def comparison(categorical, numerical, cluster_dfs):

    list_combinations = list(combinations(list(range(len(cluster_dfs))), 2))
    full_features = []
    full_pvals = []
    selected = []
    for cluster in list_combinations:
        features = []
        pvals = []
        sel = []
#         print('Clusters analyzed:', cluster)
        cluster_1 = cluster_dfs[cluster[0]]
        cluster_2 = cluster_dfs[cluster[1]]

        for i in categorical:
            len1, prop1 = cluster_1.shape[0], cluster_1.loc[cluster_1[i]==1].shape[0]/cluster_1.shape[0]
            len2, prop2 = cluster_2.shape[0], cluster_2.loc[cluster_2[i]==1].shape[0]/cluster_2.shape[0]
            pv = two_proportion_test(len1, prop1, len2, prop2, variable = i)
            features.append(i)
            pvals.append(pv)
            if pv < 0.05:
                sel.append(i)
            
        for i in numerical:
            features.append(i)
            _, pval1 = stats.shapiro(cluster_1[i])
            _, pval2 = stats.shapiro(cluster_2[i])

            if pval1 or pval2 > 0.05:
                _, pv = stats.ttest_ind(cluster_1[i], cluster_2[i]) 
                pvals.append(pv)
                if pv < 0.05:
                    sel.append(i)
            else: 
                tval, pval = stats.mannwhitneyu(cluster_1[i], cluster_2[i])
                pvals.append(pval)
                if pval < 0.05:
                    sel.append(i)
                    
        full_features.append(features)
        full_pvals.append(pvals)
        selected.append(sel)
        
    return full_features, full_pvals, selected

In [ ]:
def validation_results(X, y, classifier, shap_plot, ale_plot, shap_samples, job):

    current_time = datetime.datetime.now()
    kf = StratifiedKFold(n_splits=5, random_state = 42, shuffle=True)
    training_scores = []
    itera= 0
    test_acc = []
    test_roc = []
    test_precision = []
    test_recall = []
    test_f1 = []
    preds = []
    actual_labels = []
    
    for train_index,test_index in kf.split(X,y):
        # Generate the training and test partitions of X and Y for each iteration of CV	
        X_train, X_test = X.iloc[list(train_index)], X.iloc[list(test_index)]
        y_train, y_test = y.iloc[list(train_index)], y.iloc[list(test_index)]

        # Feature Scaling
        X_train_norm, X_test_norm, X_train_raw, X_test_raw = scale_features(X_train, X_test)
        if classifier in ['dt', 'rf', 'xgb']:
            X_train = X_train_raw
            X_test = X_test_raw
        else:
            X_train = X_train_norm
            X_test = X_test_norm
        actual_labels.extend(y_test)

        priors = np.array(y_train.value_counts()/len(y_train))

        # Get Model and Parameter grid_space
        model_classifier, param_grid_classifier, list_dict_params = get_hyperparams(classifier, num_features=X_train.shape[1], seed_value=itera)

        # Nested 5-fold CV for hyperparameter tuning and validation results
        grid_cv = GridSearchCV(estimator=model_classifier, param_grid=param_grid_classifier, scoring='accuracy', cv=5, return_train_score=True, n_jobs=-1) #
        grid_cv.fit(X_train, y_train)
        best_estimator = grid_cv.best_estimator_
        training_score = grid_cv.best_score_
        training_scores.append(training_score)
        itera= itera+ 1

        y_pred = best_estimator.predict(X_test)
        preds.extend(y_pred)
        test_acc.append(accuracy_score(y_test, y_pred))
        test_roc.append(roc_auc_score(y_test, y_pred))
        test_precision.append(precision_score(y_test, y_pred))
        test_recall.append(recall_score(y_test, y_pred))
        test_f1.append(f1_score(y_test, y_pred,average='weighted'))
        cmiter=confusion_matrix(y_test,y_pred)
        
        y_pred_proba = best_estimator.predict_proba(X_test)[::,1]
        fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
        auc = roc_auc_score(y_test, y_pred_proba)
        plt.plot(fpr,tpr,label="auc="+str(auc))
        plt.legend(loc=4)
        plt.savefig('../reports/figures/{}_{}_roc_seed_{}.pdf'.format(job, classifier, itera), bbox_inches = 'tight')
        plt.clf()
        plt.close()
        
        if shap_plot == True:
            deita = shap.sample(X_test, shap_samples) 
            explainer = shap.KernelExplainer(best_estimator.predict, deita)
            shap_values = explainer.shap_values(deita)
            shap_expected = explainer.expected_value
            shap.summary_plot(shap_values, deita, show=False)
            plt.savefig('../reports/figures/{}_{}_shap_summary_seed_{}.pdf'.format(job, classifier, itera), bbox_inches = 'tight')
            plt.clf()
            plt.close()
            shap.summary_plot(shap_values,deita, plot_type="bar", show=False)
            plt.savefig('../reports/figures/{}_{}_shap_bar_seed_{}.pdf'.format(job,classifier, itera), bbox_inches = 'tight')
            plt.clf()
            plt.close()
        
        if ale_plot == True:
            plot_ale_features(X_test, best_estimator, X_test_raw, [var for var in X_train.columns if var not in ['health_exp']], ['health_exp'], classifier, itera, job)

        metrics_dict = {'Date':current_time, 'Iteration':itera, 'Classifier':classifier,
                       'Best_params':grid_cv.best_params_, 'Training_score_iteration':grid_cv.best_score_, 
                       'accuracy_test':accuracy_score(y_test, y_pred), 'roc_test': roc_auc_score(y_test, y_pred),
                       'precision_test':precision_score(y_test, y_pred), 'recall_test':recall_score(y_test, y_pred),
                       'cm': np.asarray(cmiter),'f1': f1_score(y_test, y_pred,average='weighted')}
        
        df_metrics = pd.read_excel("../data/processed/metrics_{}.xlsx".format(job))
        df_metrics = df_metrics.append(metrics_dict, ignore_index=True)
        df_metrics.to_excel("../data/processed/metrics_{}.xlsx".format(job), index=False)

    print('Mean training score for', classifier, 'model:', np.mean(training_scores))
    print('Mean score',np.mean(test_acc), np.std(test_acc))

In [ ]:
#creating dataframes to store important training and testing metrics

metrics_carers = pd.DataFrame(columns = ['Date', 'Iteration', 'Classifier', 'Best_params','Training_score_iteration','accuracy_test','roc_test',  'precision_test','recall_test','cm','f1'])
metrics_carers.to_excel("../data/processed/metrics_carers.xlsx")

metrics_nurses = pd.DataFrame(columns = ['Date', 'Iteration', 'Classifier', 'Best_params','Training_score_iteration','accuracy_test','roc_test',  'precision_test','recall_test','cm','f1'])
metrics_nurses.to_excel("../data/processed/metrics_nurses.xlsx")

metrics_seniors = pd.DataFrame(columns = ['Date', 'Iteration', 'Classifier', 'Best_params','Training_score_iteration','accuracy_test','roc_test',  'precision_test','recall_test','cm','f1'])
metrics_seniors.to_excel("../data/processed/metrics_seniors.xlsx")



<div class="alert alert-block alert-success">
    
# CARERS
</div>

In [ ]:
carers.head()

In [ ]:
df = pd.read_csv('../data/processed/data.csv')

#mapping length of service columns for binary classification
df['class'] = np.where(df['length of service in months']>26, 1, 0)

carers = df[(df['Current Employee [1=YES / 0=No]'] == 0) & (df['Job Title'] == 0)]

X= carers.drop(['Unnamed: 0.1.1', 'Unnamed: 0.1', 'Unnamed: 0', 'Current Employee [1=YES / 0=No]',
       'Job Title'], axis = 1)

y = carers['class']

feat, pvals, selected= comparison(['health_exp'], [var for var in X.columns if var not in ['health_exp']], [X[X['class'] == 0],X[X['class'] == 1]])

In [ ]:
#checking if modified columns are there or not

if 'health_exp' not in selected[0]:
    selected[0].append('health_exp')
if 'Advertised Salary' not in selected[0]:
    selected[0].append('Advertised Salary')
    
X = X[selected[0]]

X = X.drop(['class','length of service in months','candidateid'], axis = 1)

## LR

In [ ]:
import winsound

In [ ]:
classif = 'lr'
validation_results(X, y, classif, shap_plot = True, ale_plot = True, shap_samples = 100, job = 'carers')

In [ ]:
winsound.Beep(500, 1000)

In [ ]:
#getting training and testing scores

df_metrics = pd.read_excel("../data/processed/metrics_carers.xlsx")
dfmet = df_metrics[['accuracy_test','roc_test', 'precision_test', 'recall_test','f1']][df_metrics['Classifier']=='lr']
print(dfmet.mean(axis=0))

In [ ]:
study_conf('carers','lr')

## KNN

In [ ]:
classif = 'knn'
validation_results(X, y, classif, shap_plot = True, ale_plot = True, shap_samples = 100, job = 'carers')

In [ ]:
winsound.Beep(500, 1000)

In [ ]:
df_metrics = pd.read_excel("../data/processed/metrics_carers.xlsx")
dfmet = df_metrics[['accuracy_test','roc_test', 'precision_test', 'recall_test','f1']][df_metrics['Classifier']=='knn']
print(dfmet.mean(axis=0))

In [ ]:
study_conf('carers','knn')

## RF

In [ ]:
classif = 'rf'
validation_results(X, y, classif, shap_plot = True, ale_plot = True, shap_samples = 100, job = 'carers')

In [ ]:
winsound.Beep(500, 1000)

In [ ]:
df_metrics = pd.read_excel("../data/processed/metrics_carers.xlsx")
dfmet = df_metrics[['accuracy_test','roc_test', 'precision_test', 'recall_test','f1']][df_metrics['Classifier']=='rf']
print(dfmet.mean(axis=0))

In [ ]:
study_conf('carers','rf')



<div class="alert alert-block alert-success">
    
# NURSES
</div>

In [ ]:
df = pd.read_csv('../data/processed/data.csv')

#mapping length of service columns for binary classification
df['class'] = np.where(df['length of service in months']>26, 1, 0)

carers = df[(df['Current Employee [1=YES / 0=No]'] == 0) & (df['Job Title'] == 1)]

X= carers.drop(['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'Current Employee [1=YES / 0=No]',
       'Job Title','candidateid'], axis = 1)

y = carers['class']

feat, pvals, selected= comparison(['health_exp'], [var for var in X.columns if var not in ['health_exp']], [X[X['class'] == 0],X[X['class'] == 1]])

if 'health_exp' not in selected[0]:
    selected[0].append('health_exp')
if 'Advertised Salary' not in selected[0]:
    selected[0].append('Advertised Salary')
    
X = X[selected[0]]

X = X.drop(['class','length of service in months'], axis = 1)

In [ ]:
y.hist()

## LR

In [ ]:
classif = 'lr'
validation_results(X, y, classif, shap_plot = True, ale_plot = True, shap_samples = 100, job = 'nurses')

In [ ]:
winsound.Beep(500, 1000)

In [ ]:
#getting training and testing scores

df_metrics = pd.read_excel("../data/processed/metrics_nurses.xlsx")
dfmet = df_metrics[['accuracy_test','roc_test', 'precision_test', 'recall_test','f1']][df_metrics['Classifier']=='lr']
print(dfmet.mean(axis=0))

In [ ]:
study_conf('nurses','lr')

## KNN

In [ ]:
classif = 'knn'
validation_results(X, y, classif, shap_plot = True, ale_plot = True, shap_samples = 100, job = 'nurses')

In [ ]:
winsound.Beep(500, 1000)

In [ ]:
#getting training and testing scores

df_metrics = pd.read_excel("../data/processed/metrics_nurses.xlsx")
dfmet = df_metrics[['accuracy_test','roc_test', 'precision_test', 'recall_test','f1']][df_metrics['Classifier']=='knn']
print(dfmet.mean(axis=0))

In [ ]:
study_conf('nurses','knn')

## RF

In [ ]:
classif = 'rf'
validation_results(X, y, classif, shap_plot = True, ale_plot = True, shap_samples = 100, job = 'nurses')

In [ ]:
winsound.Beep(500, 1000)

In [ ]:
#getting training and testing scores

df_metrics = pd.read_excel("../data/processed/metrics_nurses.xlsx")
dfmet = df_metrics[['accuracy_test','roc_test', 'precision_test', 'recall_test','f1']][df_metrics['Classifier']=='rf']
print(dfmet.mean(axis=0))

In [ ]:
study_conf('nurses','rf')



<div class="alert alert-block alert-success">
    
# SENIOR CARERS
</div>

In [ ]:
df = pd.read_csv('../data/processed/data.csv')

#mapping length of service columns for binary classification
df['class'] = np.where(df['length of service in months']>26, 1, 0)

carers = df[(df['Current Employee [1=YES / 0=No]'] == 0) & (df['Job Title'] == 2)]

X= carers.drop(['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'Current Employee [1=YES / 0=No]',
       'Job Title','candidateid'], axis = 1)

y = carers['class']

feat, pvals, selected= comparison(['health_exp'], [var for var in X.columns if var not in ['health_exp']], [X[X['class'] == 0],X[X['class'] == 1]])

if 'health_exp' not in selected[0]:
    selected[0].append('health_exp')
if 'Advertised Salary' not in selected[0]:
    selected[0].append('Advertised Salary')
    
X = X[selected[0]]

X = X.drop(['class','length of service in months'], axis = 1)

In [ ]:
y.hist()

## LR

In [ ]:
classif = 'lr'
validation_results(X, y, classif, shap_plot = True, ale_plot = True, shap_samples = 100, job = 'seniors')

In [ ]:
#getting training and testing scores

df_metrics = pd.read_excel("../data/processed/metrics_seniors.xlsx")
dfmet = df_metrics[['accuracy_test','roc_test', 'precision_test', 'recall_test','f1']][df_metrics['Classifier']=='lr']
print(dfmet.mean(axis=0))

In [ ]:
study_conf('seniors','lr')

## KNN

In [ ]:
classif = 'knn'
validation_results(X, y, classif, shap_plot = True, ale_plot = True, shap_samples = 100, job = 'seniors')

In [ ]:
#getting training and testing scores

df_metrics = pd.read_excel("../data/processed/metrics_seniors.xlsx")
dfmet = df_metrics[['accuracy_test','roc_test', 'precision_test', 'recall_test','f1']][df_metrics['Classifier']=='knn']
print(dfmet.mean(axis=0))
study_conf('seniors','knn')

## RF

In [ ]:
classif = 'rf'
validation_results(X, y, classif, shap_plot = True, ale_plot = True, shap_samples = 100, job = 'seniors')

In [ ]:
#getting training and testing scores

df_metrics = pd.read_excel("../data/processed/metrics_seniors.xlsx")
dfmet = df_metrics[['accuracy_test','roc_test', 'precision_test', 'recall_test','f1']][df_metrics['Classifier']=='rf']
print(dfmet.mean(axis=0))

In [ ]:
study_conf('seniors','rf')